# Model 1: meta-llama/Meta-Llama-3-8B-Instruct

In [1]:
!pip install -U accelerate transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.5 MB/s eta 0:00:00


In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00


In [3]:
!pip install accelerate --q

In [4]:
!pip install -U datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [ ]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoConfig, BitsAndBytesConfig

In [2]:
q_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_quant_type="nf4",
  )

In [3]:
def create_model_tokenizer(name, q_config):
  tokenizer = AutoTokenizer.from_pretrained(name)
  model = AutoModelForCausalLM.from_pretrained (
      name,
      quantization_config = q_config,
      device_map="cuda",
      torch_dtype="auto",
      trust_remote_code = True
  )
  return model, tokenizer

In [4]:
# Load model directly
model1, tokenizer1 = create_model_tokenizer("NousResearch/Meta-Llama-3-8B-Instruct", q_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Zero-shot Learning

In [5]:
generation_pipeline = pipeline(
    "text-generation",
    model=model1,
    tokenizer=tokenizer1,
    framework="pt",
)

In [6]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [7]:
question1 = "What is the capital of France?"
question2 = "Which city is the capital of France?"
# question1 = "How old are you?"
# question2 = "What is your age?"
prompt1 = f"Question 1: {question1}\nQuestion 2: {question2}\nAre these questions similar? Just say 'yes' or 'no'"

In [8]:
messages1 = [
    {"role": "user", "content": prompt1},
]

In [9]:
pipe1 = pipeline(
    "text-generation",
    model=model1,
    tokenizer=tokenizer1,
)

In [10]:
output1 = pipe1(messages1, **generation_args)
print(output1[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Yes


## One-shot Learning

In [11]:
from datasets import load_dataset
import random

In [12]:
dataset = load_dataset("persiannlp/parsinlu_entailment")
# for x in dataset['train']:
#   print(x)

Generating train split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1675 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/270 [00:00<?, ? examples/s]

In [13]:
dataset["train"][0]

{'sent1': 'زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد.',
 'sent2': 'مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.',
 'category': 'translation-train',
 'label': 'c'}

In [14]:
random.choice(dataset["train"])

{'sent1': 'یک غواص باتجربه ممکن است حرکت به سمت سینا را ترجیح دهد ، جایی که غواصی حتی بهتر است.',
 'sent2': 'تجربه غواصی در سینا بهتر به حساب می آید.',
 'category': 'translation-train',
 'label': 'e'}

In [15]:
random.choice(dataset["test"])

{'sent1': 'سایر تمایزها مانند فله\u200cای و غیرفله\u200cای امکان\u200cپذیر است، اما به نظر می رسد که تمایز در سرعت تعیین\u200cکننده است.',
 'sent2': 'سایر تمایزها ممکن نیست،',
 'category': 'translation-train',
 'label': 'c'}

In [16]:
train = random.choice(dataset["train"])
test = random.choice(dataset["test"])
# question1 = "How old are you?"
# question2 = "What is your age?"
prompt2 = f"Sentence 1: {train['sent1']}\nSentence 2: {train['sent2']}\nThe label for these 2 questions is:{train['label']}\nNow tell me this: Are these questions similar? Just say 'yes' or 'no'\n Sentence 3: {test['sent1']}\n Sentence 4: {test['sent2']}"

In [17]:
messages2 = [
    {"role": "user", "content": prompt2},
]

In [18]:
output2 = pipe1(messages2, **generation_args)
print(output2[0]['generated_text'])
print(test['label'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The label for these questions is: Persian (Farsi) sentences.

And, yes, sentences 1 and 2 are similar, as they are both asking a question.
c


## Five-shot Learning

In [29]:
fiveshot_train = dataset["train"].select(range(5))
fiveshot_test = dataset["test"].select(range(5))

In [30]:
fiveshot_train

Dataset({
    features: ['sent1', 'sent2', 'category', 'label'],
    num_rows: 5
})

In [31]:
prompt3 = (
          f"Sentence 1: {fiveshot_train[0]['sent1']}\nSentence 2: {fiveshot_train[0]['sent2']}\nThe label for these 2 questions is:{fiveshot_train[0]['label']};\n"
          f"Sentence 3: {fiveshot_train[1]['sent1']}\nSentence 4: {fiveshot_train[1]['sent2']}\nThe label for these 2 questions is:{fiveshot_train[1]['label']};\n"
          f"Sentence 5: {fiveshot_train[2]['sent1']}\nSentence 6: {fiveshot_train[2]['sent2']}\nThe label for these 2 questions is:{fiveshot_train[2]['label']};\n"
          f"Sentence 7: {fiveshot_train[3]['sent1']}\nSentence 8: {fiveshot_train[3]['sent2']}\nThe label for these 2 questions is:{fiveshot_train[3]['label']};\n"
          f"Sentence 9: {fiveshot_train[4]['sent1']}\nSentence 10: {fiveshot_train[4]['sent2']}\nThe label for these 2 questions is:{fiveshot_train[4]['label']};\n"
          f"Now tell me this: Are these questions similar? Just say 'yes' or 'no'\n Sentence 3: {fiveshot_test[0]['sent1']}\n Sentence 4: {fiveshot_test[0]['sent2']}"
          )

In [32]:
messages3 = [
    {"role": "user", "content": prompt3},
]

In [33]:
output3 = pipe1(messages3, **generation_args)
print(output3[0]['generated_text'])
print(fiveshot_test[0]['label'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the sentences you provided, I would say that the questions are similar.
c


# Model 2: ViraIntelligentDataMining/PersianLLaMA-13B-Instruct

In [ ]:
model2, tokenizer2 = create_model_tokenizer("ViraIntelligentDataMining/PersianLLaMA-13B", q_config)

tokenizer_config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

## Zero-shot Learning

In [ ]:
generation_pipeline2 = pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer2,
    framework="pt",
)

In [ ]:
pipe2 = pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer2,
)